In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import argparse
import importlib
import sys;sys.argv=[''] #passes no cmd line arguments
sys.path.append(os.path.join(os.getcwd(),'RAVEN/src/model'))
sys.path.append(os.path.join(os.getcwd(),'RAVEN/src/model/utility/'))

from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
#setup argparser
if 1:
    parser=argparse.ArgumentParser()
    parser.add_argument('--model', type=str, default='Resnet18_MLP')
    parser.add_argument('--epochs', type=int, default=200)
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--seed', type=int, default=12345)
    parser.add_argument('--device', type=int, default=0)
    parser.add_argument('--load_workers', type=int, default=16)
    parser.add_argument('--resume', type=bool, default=False)
    parser.add_argument('--path', type=str, default='/Users/iyngkarrankumar/Documents/AI/AVR-functional-modularity/datasets/RAVEN-10000')
    parser.add_argument('--save', type=str, default='./experiments/checkpoint/')
    parser.add_argument('--img_size', type=int, default=224)
    parser.add_argument('--lr', type=float, default=1e-4)
    parser.add_argument('--beta1', type=float, default=0.9)
    parser.add_argument('--beta2', type=float, default=0.999)
    parser.add_argument('--epsilon', type=float, default=1e-8)
    parser.add_argument('--meta_alpha', type=float, default=0.0)
    parser.add_argument('--meta_beta', type=float, default=0.0)

args=parser.parse_args()

In [ ]:
#load img
#to load whole dataset will need to write a torch Dataset class

import data; importlib.reload(data)
batch_size=8

datamodule=data.IRAVENDataModule(batch_size=batch_size)
datamodule.setup()
train_dataloader,val_dataloader,test_dataloader=datamodule.train_dataloader(),datamodule.val_dataloader(),datamodule.test_dataloader()

image,target,meta_target,meta_structure=next(iter(train_dataloader))


In [ ]:
#model
import models
importlib.reload(models)

model=models.CNN_MLP()
out=model(image)